In [1]:
#Inisialisasi Library

from PIL import Image
from PIL import ImageDraw

import glob
import pandas as pd
from skimage.feature import greycomatrix, greycoprops
from skimage.transform import resize
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import math

ubah8bit_3bit = [i // 32 for i in range(256)]


Bad key "text.kerning_factor" on line 4 in
C:\Users\ASUS\Anaconda3\envs\tensorflowproject\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
# Fungsi-fungsi
# Proses membuka gambar

def read_file(mypath):
    file_name = []
    label = []
    full_path = []

    for path, subdirs, files in os.walk(mypath):
        for name in files:
            full_path.append(os.path.join(path, name))
            label.append(path.split('/')[-1])
            
    return full_path, label

def load_img(img_path):
    img = Image.open(img_path).convert('L')
    hasil_img = img.resize((224,224))
    return hasil_img

def print_img(img):
    plt.imshow(img, cmap='gray')
    
def getLabels(path):
    label = os.listdir(path)
    return label
    
# Preprocessing Gambar
    
def getAllColorIMG(img):
    AllColorIMG = list(img.getdata())
    return AllColorIMG

def ubahKe3bit(AllColorIMG):
    i = 0
    
    for ColorIMG in AllColorIMG:
        AllColorIMG[i] = ubah8bit_3bit[ColorIMG]
        i += 1
        
    return AllColorIMG

def ubahKeMatriks(AllColorIMG):
    matriksIMG = [] 
    i = 0
    
    dimensiGambar = int(math.sqrt(len(AllColorIMG)))
    
    for row in range(dimensiGambar):
        newRowMat = []
        for col in range(dimensiGambar):
            newRowMat.append(AllColorIMG[i])
            i += 1
        matriksIMG.append(newRowMat)
        
    return matriksIMG

def featureExtract(img):
    colorIMG = getAllColorIMG(img)
    _3BitColor = ubahKe3bit(colorIMG)
    matriksIMG = ubahKeMatriks(_3BitColor)
    
    resultOfGLCM = []
    glcm = greycomatrix(matriksIMG, distances=[1], angles=[0], levels=256, symmetric=False, normed=False)
    
    komponen = ['dissimilarity', 'contrast', 'homogeneity', 'energy', 'correlation', 'ASM']
    
    for i in komponen:
        resultOfGLCM.append(greycoprops(glcm, i)[0][0])
    
    return resultOfGLCM

In [3]:
# Memproses File Training dengan mengekstrak fiturnya sebelum dimodelkan

train_path = 'dataset_magang/Training/'
imgType = '*.jpg'

PathOf_trainIMG = []
LabelOf_trainIMG = []
Dissimilarity = []
Contrast = []
Homogeneity = []
Energy = []
Correlation = []
ASM = []

labels = getLabels(train_path)

for label in labels:
    full_paths = glob.glob(train_path + label + '/' + imgType)
    for full_path in full_paths:
        PathOf_trainIMG.append(full_path)
        LabelOf_trainIMG.append(label)
        trainIMG = load_img(full_path)
        featureOfIMG = featureExtract(trainIMG)
        Dissimilarity.append(featureOfIMG[0])
        Contrast.append(featureOfIMG[1])
        Homogeneity.append(featureOfIMG[2])
        Energy.append(featureOfIMG[3])
        Correlation.append(featureOfIMG[4])
        ASM.append(featureOfIMG[5])

columns = ["Path", "Dissimilarity", "Contrast", "Homogeneity", "Energy", "Correlation", "ASM" , "Label"]
variables = {"Path":PathOf_trainIMG, "Dissimilarity":Dissimilarity, "Contrast":Contrast, "Homogeneity":Homogeneity, "Energy":Energy, "Correlation":Correlation, "ASM":ASM ,"Label":LabelOf_trainIMG}
df_train = pd.DataFrame(variables, columns = columns)
df_train.to_csv('DataTrainingBatik.csv', index=False)     

In [4]:
df_train

X = df_train.iloc[:, 1:6].values  
y = df_train.iloc[:, 7].values

print(X, "\n")
print(y)

[[0.39455878 0.5490671  0.81771979 0.54579403 0.60809678]
 [0.51012972 0.72109225 0.76569344 0.5179088  0.38704864]
 [0.49805814 0.69873078 0.77092363 0.49609956 0.66136359]
 ...
 [0.43978219 0.61619154 0.79773288 0.49234607 0.60026921]
 [0.42432735 0.60434017 0.80579238 0.51809596 0.5069144 ]
 [0.42604901 0.59012652 0.80337477 0.52303746 0.49599562]] 

['Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap'
 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap'
 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap'
 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap'
 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap'
 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap'
 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap'
 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap'
 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap' 'Cap'

# <center>Metode Decision Tree<center>

In [5]:
#Proses Modelling Data Training

from sklearn.model_selection import train_test_split

X = df_train.iloc[:, 1:6].values  
y = df_train.iloc[:, 7].values
X_trainDT, X_testDT, y_trainDT, y_testDT = train_test_split(X, y, test_size=0.3, random_state=0)

In [6]:
# Melakukan klasifikasi dengan decision tree kemudian menetapkan model decision tree dengan data training

from sklearn.tree import DecisionTreeClassifier

classifierDT = DecisionTreeClassifier(criterion="entropy", max_depth=4)
classifierDT.fit(X_trainDT, y_trainDT)
y_predDT = classifierDT.predict(X_testDT)

In [7]:
# Menampilkan confusion matrix dan classification_report untuk melihat performasi algoritma melalui recall, presisi,F1-Score,dll

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

print("Algoritma Decision Tree")
print("Confusion Matrix:\n", confusion_matrix(y_testDT, y_predDT), "\n")
print("Classification Measure\n", classification_report(y_testDT, y_predDT))
print("Accuracy:",accuracy_score(y_testDT, y_predDT))

Algoritma Decision Tree
Confusion Matrix:
 [[69 13]
 [14 65]] 

Classification Measure
               precision    recall  f1-score   support

         Cap       0.83      0.84      0.84        82
       Tulis       0.83      0.82      0.83        79

    accuracy                           0.83       161
   macro avg       0.83      0.83      0.83       161
weighted avg       0.83      0.83      0.83       161

Accuracy: 0.8322981366459627


# <center>Metode k-NN<center>

In [8]:
X_trainKNN, X_testKNN, y_trainKNN, y_testKNN = train_test_split(X, y, test_size=0.3, random_state=0)

from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
scaler.fit(X_trainKNN)

X_trainKNN = scaler.transform(X_trainKNN)  
X_testKNN = scaler.transform(X_testKNN)

#Melakukan klasifikasi dengan k neighbors=5
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(n_neighbors=5)  
classifier.fit(X_trainKNN, y_trainKNN)

#Melakukan prediksi dari klasifikasi
y_predKNN = classifier.predict(X_testKNN)

#Menampilkan confussion matriks guna melihat recall, presisi, F!Score, dll
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print("Algoritma k-NN")
print ("Confusion Matrix:\n", confusion_matrix(y_testKNN, y_predKNN), "\n")
print ("Classification Measure\n", classification_report(y_testKNN, y_predKNN))
print("Accuracy:",accuracy_score(y_testKNN, y_predKNN))

Algoritma k-NN
Confusion Matrix:
 [[72 10]
 [10 69]] 

Classification Measure
               precision    recall  f1-score   support

         Cap       0.88      0.88      0.88        82
       Tulis       0.87      0.87      0.87        79

    accuracy                           0.88       161
   macro avg       0.88      0.88      0.88       161
weighted avg       0.88      0.88      0.88       161

Accuracy: 0.8757763975155279


# <center>Metode Naive Bayes

In [9]:
from sklearn.naive_bayes import GaussianNB

X_trainNB, X_testNB, y_trainNB, y_testNB = train_test_split(X, y, test_size=0.3, random_state=0)

#membuat Gaussian Classifier kemudian menetapkan model Gaussian Classifier menggunakan set training
gnb = GaussianNB()
gnb.fit(X_trainNB, y_trainNB)
y_predNB = gnb.predict(X_testNB) #memprediksi respon dari dataset test

print ("Algoritma Naive Bayes")
print ("Confusion Matrix:\n", confusion_matrix(y_testNB, y_predNB), "\n")
print ("Classification Measure\n", classification_report(y_testNB, y_predNB))
print("Accuracy:",accuracy_score(y_testNB, y_predNB))

Algoritma Naive Bayes
Confusion Matrix:
 [[46 36]
 [13 66]] 

Classification Measure
               precision    recall  f1-score   support

         Cap       0.78      0.56      0.65        82
       Tulis       0.65      0.84      0.73        79

    accuracy                           0.70       161
   macro avg       0.71      0.70      0.69       161
weighted avg       0.71      0.70      0.69       161

Accuracy: 0.6956521739130435


# <center>Metode Support Vector Machines (SVMs)

In [10]:
from sklearn import svm
from sklearn.svm import SVC  


kernels = ['Polynomial', 'RBF', 'Sigmoid','Linear']
#A function which returns the corresponding SVC model
def getClassifier(ktype):
    if ktype == 0:
        # Polynomial kernal
        return SVC(kernel='poly', degree=8, gamma="auto")
    elif ktype == 1:
        # Radial Basis Function kernal
        return SVC(kernel='rbf', gamma="auto")
    elif ktype == 2:
        # Sigmoid kernal
        return SVC(kernel='sigmoid', gamma="auto")
    elif ktype == 3:
        # Linear kernal
        return SVC(kernel='linear', gamma="auto")

for i in range(4):
    # Separate data into test and training sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=0)
# Train a SVC model using different kernal
    svclassifier = getClassifier(i) 
    svclassifier.fit(X_train, y_train)
# Make prediction
    y_pred = svclassifier.predict(X_test)
# Evaluate our model
    print("Evaluation:", kernels[i], "kernel")
    print(classification_report(y_test,y_pred))
    print ("Accuracy:",accuracy_score(y_test, y_pred))


Evaluation: Polynomial kernel
              precision    recall  f1-score   support

         Cap       1.00      0.26      0.41        82
       Tulis       0.56      1.00      0.72        79

    accuracy                           0.62       161
   macro avg       0.78      0.63      0.56       161
weighted avg       0.79      0.62      0.56       161

Accuracy: 0.6211180124223602
Evaluation: RBF kernel
              precision    recall  f1-score   support

         Cap       0.89      0.51      0.65        82
       Tulis       0.65      0.94      0.77        79

    accuracy                           0.72       161
   macro avg       0.77      0.72      0.71       161
weighted avg       0.77      0.72      0.71       161

Accuracy: 0.7204968944099379
Evaluation: Sigmoid kernel
              precision    recall  f1-score   support

         Cap       0.40      0.50      0.45        82
       Tulis       0.31      0.23      0.26        79

    accuracy                           0.37 

In [15]:
# Mengimport library yang akan digunakan untuk melakukan variasi parameter dengan Grid Search
from sklearn.model_selection import GridSearchCV

#Create a dictionary called param_grid and fill out some parameters for kernels, C and gamma
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001]}

# Membangun model dengan grid search
grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=2)
grid.fit(X_train,y_train)

print(grid.best_estimator_)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] C=0.1, gamma=1 ..................................................
[CV] ................................... C=0.1, gamma=1, total=   0.0s
[CV] C=0.1, gamma=1 ..................................................
[CV] ................................... C=0.1, gamma=1, total=   0.0s
[CV] C=0.1, gamma=1 ..................................................
[CV] ................................... C=0.1, gamma=1, total=   0.1s
[CV] C=0.1, gamma=1 ..................................................
[CV] ................................... C=0.1, gamma=1, total=   0.0s
[CV] C=0.1, gamma=1 ..................................................
[CV] ................................... C=0.1, gamma=1, total=   0.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] ................................. C=0.1, gamma=0.1, total=   0.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] ...........

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s



[CV] C=0.1, gamma=0.001 ..............................................
[CV] ............................... C=0.1, gamma=0.001, total=   0.0s
[CV] C=0.1, gamma=0.001 ..............................................
[CV] ............................... C=0.1, gamma=0.001, total=   0.0s
[CV] C=0.1, gamma=0.001 ..............................................
[CV] ............................... C=0.1, gamma=0.001, total=   0.0s
[CV] C=0.1, gamma=0.001 ..............................................
[CV] ............................... C=0.1, gamma=0.001, total=   0.0s
[CV] C=0.1, gamma=0.001 ..............................................
[CV] ............................... C=0.1, gamma=0.001, total=   0.0s
[CV] C=1, gamma=1 ....................................................
[CV] ..................................... C=1, gamma=1, total=   0.0s
[CV] C=1, gamma=1 ....................................................
[CV] ..................................... C=1, gamma=1, total=   0.0s
[CV] 

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    1.1s finished


In [16]:
print(grid.best_estimator_)

SVC(C=100, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
    probability=False, random_state=None, shrinking=True, tol=0.001,
    verbose=False)


In [17]:
# Menerima respon hasil prediksi
grid_pred = grid.predict(X_test)
# Menampilkan confusion matrix dan classification_report untuk melihat performasi algoritma melalui recall, presisi,F1-Score,dll
print ("Confusion Matrix:\n", confusion_matrix(y_test, grid_pred), "\n")
print ("Classification Measure\n", classification_report(y_test, grid_pred))
print ("Accuracy:",accuracy_score(y_test, grid_pred))

Confusion Matrix:
 [[72 10]
 [10 69]] 

Classification Measure
               precision    recall  f1-score   support

         Cap       0.88      0.88      0.88        82
       Tulis       0.87      0.87      0.87        79

    accuracy                           0.88       161
   macro avg       0.88      0.88      0.88       161
weighted avg       0.88      0.88      0.88       161

Accuracy: 0.8757763975155279


# <center>Metode Random Forest

In [18]:
from sklearn.ensemble import RandomForestClassifier

X_trainRF, X_testRF, y_trainRF, y_testRF = train_test_split(X, y, test_size=0.3, random_state=0)

# Membangun model dengan Random Forest Classifier dengan data training
model = RandomForestClassifier(n_estimators=100, random_state=0)
model.fit(X_trainRF, y_trainRF)
y_predRF = model.predict(X_testRF)

# Menampilkan confusion matrix dan classification_report untuk melihat performasi algoritma melalui recall, presisi,F1-Score,dll
print ("Algoritma Random Forest")
print ("Confusion Matrix:\n", confusion_matrix(y_testRF, y_predRF), "\n")
print ("Classification Measure\n", classification_report(y_testRF, y_predRF))
print ("Accuracy:",accuracy_score(y_testRF, y_predRF))

Algoritma Random Forest
Confusion Matrix:
 [[72 10]
 [ 6 73]] 

Classification Measure
               precision    recall  f1-score   support

         Cap       0.92      0.88      0.90        82
       Tulis       0.88      0.92      0.90        79

    accuracy                           0.90       161
   macro avg       0.90      0.90      0.90       161
weighted avg       0.90      0.90      0.90       161

Accuracy: 0.9006211180124224
